In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import matplotlib.backends.backend_pdf

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

%matplotlib inline

dir_data='/Users/ahenny/'
dir1='/Volumes/LaCie/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/day/'

In [ ]:
#yrs=np.arange(1983,2023,1)
yrs=np.arange(1980,2017,1)
#yrs=np.arange(1997,2023,1)
#yrs=np.arange(1950,2017,1)
#yrs=np.arange(1982,2021,1)
#yrs=np.arange(1979,2024,1)
#yrs=np.arange(2001,2023,1)
#yrs=np.arange(1940,2023,1)
#yrs=np.arange(1958,2023,1)
#yrs=np.arange(1980,2023,1)

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    #ds=xr.open_dataset(dir_data+'holding/persiann_cdr_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset(dir2+'mswep_past_deres_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset(dir_data+'holding/'+'gpcp_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_1950-2016_QualityMask.nc')
    #quality=ds['p']
    ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_'+str(year)+'.nc')
    precip=ds['p']#.where(quality==1)
    
    #ds=xr.open_dataset('/Users/ahenny/Downloads/full_data_daily_v2022_10_'+str(year)+'.nc')
    #precip=ds['precip'].transpose('time','lat','lon')
    
    #ds=xr.open_dataset(dir2+'precip.'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset('/Volumes/My Passport/day/imerg_combined_deres_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset(dir_data+'holding/era_5_precip_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset(dir2+'jra55_precip_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset(dir_data+'holding/merra2_precip_'+str(year)+'.nc')
    #precip=ds['precip']
    
    if 1==0:#PERSIANN-CDR, CPC
        lons_east=[x for x in precip.lon.values if x>=180.]
        lons_west=[x for x in precip.lon.values if x<180.]
        p_east=precip.sel(lon=lons_east)
        p_west=precip.sel(lon=lons_west)
        p_east['lon']=[x-360. for x in lons_east]
        precip=xr.concat([p_east,p_west],dim='lon')
        p_east.close()
        p_west.close()
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    
    if i==0:
        max_concat=defined1
        present_concat=defined
    else:
        max_concat=xr.concat([max_concat,defined1],dim='time')
        present_concat=xr.concat([present_concat,defined],dim='time')

In [ ]:
max_all=max_concat.sum(dim='time')
present_all=present_concat.sum(dim='time')
ratio_all=present_all/max_all

In [ ]:
#yrs=np.arange(1983,2023,1)
yrs=np.arange(1980,2017,1)
#yrs=np.arange(1997,2023,1)
#yrs=np.arange(1950,2017,1)
#yrs=np.arange(1982,2021,1)
#yrs=np.arange(1979,2024,1)
#yrs=np.arange(2001,2023,1)
#yrs=np.arange(1940,2023,1)
#yrs=np.arange(1958,2023,1)
#yrs=np.arange(1980,2023,1)

lats_sel=[x for x in ratio_all.lat.values if -60<=x<=70]
ratio_all=ratio_all.sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'persiann_threshold_r95p.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'mswep_threshold_r95p.nc')
#ds=xr.open_dataset(dir_data+'mswep_threshold_final_98_all_30.nc')
#threshold=ds['threshold']#.sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'gpcp_threshold_r95p.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

ds=xr.open_dataset(dir_data+'regenfull_threshold_r95p.nc')
threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'gpcc_threshold_r95p.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'cpc_threshold_r95p.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir3+'imerg_threshold_r95p.nc')
#threshold=ds['threshold']

#ds=xr.open_dataset(dir_data+'era5_threshold_r95p.nc')
#threshold=ds['threshold']#.sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'jra55_threshold_r95p.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

#ds=xr.open_dataset(dir_data+'merra2_threshold_r95p.nc')
#threshold=ds['threshold'].sel(lat=lats_sel)

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    #ds=xr.open_dataset(dir_data+'holding/persiann_cdr_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset(dir2+'mswep_past_deres_'+str(year)+'.nc')
    #precip=ds['precip']#.sel(lat=lats_sel)
    
    #ds=xr.open_dataset(dir_data+'holding/'+'gpcp_'+str(year)+'.nc')
    #precip=ds['precip'].sel(lat=lats_sel)
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_'+str(year)+'.nc')
    precip=ds['p'].sel(lat=lats_sel)
    
    #ds=xr.open_dataset('/Users/ahenny/Downloads/full_data_daily_v2022_10_'+str(year)+'.nc')
    #precip=ds['precip'].transpose('time','lat','lon').sel(lat=lats_sel)
    
    #ds=xr.open_dataset(dir2+'precip.'+str(year)+'.nc')
    #precip=ds['precip'].sel(lat=lats_sel)
    
    #ds=xr.open_dataset('/Volumes/My Passport/day/imerg_combined_deres_'+str(year)+'.nc')
    #precip=ds['precip']
    
    #ds=xr.open_dataset(dir_data+'holding/era_5_precip_'+str(year)+'.nc')
    #lats_sel=[x for x in ds.lat.values if -60<=x<=70]
    #precip=ds['precip'].sel(lat=lats_sel)
    #threshold=threshold.sel(lat=lats_sel)
    
    #ds=xr.open_dataset(dir2+'jra55_precip_'+str(year)+'.nc')
    #precip=ds['precip'].sel(lat=lats_sel)
    
    #ds=xr.open_dataset(dir_data+'holding/merra2_precip_'+str(year)+'.nc')
    #precip=ds['precip'].sel(lat=lats_sel)
    
    if 1==0:
        lons_east=[x for x in precip.lon.values if x>=180.]
        lons_west=[x for x in precip.lon.values if x<180.]
        p_east=precip.sel(lon=lons_east)
        p_west=precip.sel(lon=lons_west)
        p_east['lon']=[x-360. for x in lons_east]
        precip=xr.concat([p_east,p_west],dim='lon')
        p_east.close()
        p_west.close()

    if 1==0:
        threshold_new=xr.zeros_like(precip)
        for k in range(threshold_new.time.size):
            date=pd.to_datetime(threshold_new.time.values[k])
            if date.month==2 and date.day==29:
                threshold_new1=threshold.sel(time=dt.datetime(1981,2,28))
                threshold_new2=threshold.sel(time=dt.datetime(1981,3,1))
                threshold_new3=(threshold_new1+threshold_new1)/2.
            else:
                threshold_new3=threshold.sel(time=dt.datetime(1981,date.month,date.day))

            threshold_new[k,:,:]=threshold_new3.values
    
    if 1==1:
        threshold_new=threshold
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    ratio=defined/defined1
    ratio=ratio.where(ratio>0)
    
    precip_sel=precip.where(precip>=1.)
    ones_sel=xr.ones_like(precip).where(precip>=1.)
    
    precip_sel=precip_sel.where(precip>=threshold_new)
    ones_sel=ones_sel.where(precip>=threshold_new)
    
    precip_total=precip_sel.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all>=0.9)/ratio
    freq_total=ones_sel.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all>=0.9)/ratio
    intensity_sel=precip_sel.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all>=0.9)
    #precip_total=precip_sel.sum(dim='time',skipna=True).where(ratio>=0.9)/ratio
    #freq_total=ones_sel.sum(dim='time',skipna=True).where(ratio>=0.9)/ratio
    #intensity_sel=precip_sel.mean(dim='time',skipna=True).where(ratio>=0.9)
    
    if i==0:
        total_concat=precip_total
        freq_concat=freq_total
        intensity_concat=intensity_sel
    else:
        total_concat=xr.concat([total_concat,precip_total],dim='time')
        freq_concat=xr.concat([freq_concat,freq_total],dim='time')
        intensity_concat=xr.concat([intensity_concat,intensity_sel],dim='time')

In [ ]:
print(intensity_concat)

In [ ]:
#total_concat=total_concat.where(ratio_all>=0.9)
#freq_concat=freq_concat.where(ratio_all>=0.9)
#intensity_concat=intensity_concat.where(ratio_all>=0.9)

slopes_array_total=xr.zeros_like(total_concat[0,:,:])
sigs_array_total=xr.zeros_like(total_concat[0,:,:])
slopes_array_freq=xr.zeros_like(freq_concat[0,:,:])
sigs_array_freq=xr.zeros_like(freq_concat[0,:,:])
slopes_array_intensity=xr.zeros_like(intensity_concat[0,:,:])
sigs_array_intensity=xr.zeros_like(intensity_concat[0,:,:])
for j in range(freq_concat.lat.size):
    print(j)
    for k in range(freq_concat.lon.size):
        
        time_series=total_concat[:,j,k].values.tolist()
        zipped=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped if x[1]>=0]
        select_values=[x[1] for x in zipped if x[1]>=0]
        
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slope=r[0]
            p_value=r[3]
        else:
            slope=np.nan
            p_value=np.nan
            
        slopes_array_total[j,k]=slope
        sigs_array_total[j,k]=p_value
        
        
        time_series=freq_concat[:,j,k].values.tolist()
        zipped=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped if x[1]>=0]
        select_values=[x[1] for x in zipped if x[1]>=0]
        
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slope=r[0]
            p_value=r[3]
        else:
            slope=np.nan
            p_value=np.nan
            
        slopes_array_freq[j,k]=slope
        sigs_array_freq[j,k]=p_value
        
        
        time_series=intensity_concat[:,j,k].values.tolist()
        zipped=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped if x[1]>=0]
        select_values=[x[1] for x in zipped if x[1]>=0]
        
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slope=r[0]
            p_value=r[3]
        else:
            slope=np.nan
            p_value=np.nan
            
        slopes_array_intensity[j,k]=slope
        sigs_array_intensity[j,k]=p_value

In [ ]:
print(N)

In [ ]:
p_values=sigs_array_total.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
a=0.05#/sum_factor
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_total=max(pass_list)
print(p_final_total)

p_values=sigs_array_freq.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_freq=max(pass_list)
print(p_final_freq)

p_values=sigs_array_intensity.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_intensity=max(pass_list)
print(p_final_intensity)

In [ ]:
field=slopes_array_total
field1=sigs_array_total
mean=total_concat.mean(dim='time',skipna=True)

field=field/mean*100.

from matplotlib.colors import ListedColormap
cmap1=plt.cm.seismic(np.linspace(0.2,0.8,128))
cmap_new=ListedColormap(cmap1)

fig=plt.figure(figsize=(24,7))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-1.5,vmax=1.5,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15)

X=2#3
#X=11
for j in range(int(field1.lat.size/X)):
    for k in range(int(field1.lon.size/X)):
        if field1[X*j,X*k].values.tolist()<=p_final_total:
            #ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3.5)
            ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3)
        #elif field1[X*j,X*k].values.tolist()<=0.05:
        #    ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='w',markersize=1.75)#1.5

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=26) 
cbar.set_label('% yr$^{-1}$',fontsize=27,labelpad=5)
ax.coastlines(resolution='10m',linewidth=2)
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':24,'color':'k'}
g1.ylabel_style={'size':24,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
ax.set_ylim(-60,70)
#ax.set_ylim(-89.5,89.5)
            
ax.set_title(r'$\bf{R95p}$',fontsize=46,pad=0)
#ax.set_title(r'$\bf{R1mm}$',fontsize=46,pad=0)
#ax.set_title(r'$\bf{R98p_{all}}$',fontsize=40,pad=0)
#ax.set_title(r'$\bf{R1mm}$',fontsize=40,pad=0)
plt.show()

In [ ]:
#fig.savefig(dir_data+'geogrid_1_merra2_r95p.png')
fig.savefig(dir_data+'geogrid_paper_supp_regen_1.png')

In [ ]:
a=sigs_array_freq/sigs_array_freq
b=a.where(sigs_array_freq<=0.05)
print(a.sum().values)
print(b.sum().values)

In [ ]:
field=slopes_array_freq
field1=sigs_array_freq
mean=freq_concat.mean(dim='time',skipna=True)

field=field/mean*100.

from matplotlib.colors import ListedColormap
cmap1=plt.cm.seismic(np.linspace(0.2,0.8,128))
cmap_new=ListedColormap(cmap1)

fig=plt.figure(figsize=(24,7))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-1.5,vmax=1.5,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15)

X=2
count=0
for j in range(int(field1.lat.size/X)):
    for k in range(int(field1.lon.size/X)):
        if field1[X*j,X*k].values.tolist()<=p_final_freq:
            count=count+1
            ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3)
        #elif field1[X*j,X*k].values.tolist()<=0.05:
        #    ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='w',markersize=1.75)
print(count)
g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=26) 
cbar.set_label('% yr$^{-1}$',fontsize=27,labelpad=5)
ax.coastlines(resolution='10m',linewidth=2)
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':24,'color':'k'}
g1.ylabel_style={'size':24,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
ax.set_ylim(-60,70)
#ax.set_ylim(-89.5,89.5)
            
ax.set_title(r'$\bf{F95p}$',fontsize=46,pad=0)
#ax.set_title(r'$\bf{F1mm}$',fontsize=46,pad=0)
#ax.set_title(r'$\bf{F98p_{all}}$',fontsize=40,pad=0)
#ax.set_title(r'$\bf{F1mm}$',fontsize=40,pad=0)
plt.show()

In [ ]:
#fig.savefig(dir_data+'geogrid_2_merra2_r95p.png')
fig.savefig(dir_data+'geogrid_paper_supp_regen_2.png')

In [ ]:
field=slopes_array_intensity
field1=sigs_array_intensity
mean=intensity_concat.mean(dim='time',skipna=True)

field=field/mean*100.

from matplotlib.colors import ListedColormap
cmap1=plt.cm.seismic(np.linspace(0.2,0.8,128))
cmap_new=ListedColormap(cmap1)

fig=plt.figure(figsize=(24,7))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-1.5,vmax=1.5,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15)

X=2
for j in range(int(field1.lat.size/X)):
    for k in range(int(field1.lon.size/X)):
        if field1[X*j,X*k].values.tolist()<=p_final_intensity:
            ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3)
        #elif field1[X*j,X*k].values.tolist()<=0.05:
        #    ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='w',markersize=1.75)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=26) 
cbar.set_label('% yr$^{-1}$',fontsize=27,labelpad=5)
ax.coastlines(resolution='10m',linewidth=2)
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':24,'color':'k'}
g1.ylabel_style={'size':24,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
ax.set_ylim(-60,70)
#ax.set_ylim(-89.5,89.5)
            
ax.set_title(r'$\bf{I95p}$',fontsize=46,pad=0)
#ax.set_title(r'$\bf{I1mm}$',fontsize=46,pad=0)
#ax.set_title(r'$\bf{I98p_{all}}$',fontsize=40,pad=0)
#ax.set_title(r'$\bf{I1mm}$',fontsize=40,pad=0)
plt.show()

In [ ]:
#fig.savefig(dir_data+'geogrid_3_merra2_r95p.png')
fig.savefig(dir_data+'geogrid_paper_supp_regen_3.png')